In [262]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import PublicDataReader as pdr
# 애플코딕 설치되어 있을때
plt.rcParams['font.family'] = 'AppleGothic'

# 전처리

## 전기차수

In [263]:
#데이터 불러오기 
car_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/데이터_행정동별 자동차 등록현황_법정동변환_법정동수정(상일,일원).csv')
car_df = car_data.copy()
car_df['전기차비율'] = ((car_df['전기'] / car_df['합계'] )* 100).round(3)
car_df = car_df[['행정동코드','자치구', '행정동', '전기','합계', '전기차비율']]
car_df = car_df.rename(columns = {'합계' : '등록차대수', '전기' : '전기차수'})
car_df = car_df[car_df['전기차수']!=0]
display(car_df)

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율
0,1168066000,강남구,개포1동,234,10434,2.243
1,1168067000,강남구,개포2동,443,15656,2.830
2,1168067500,강남구,개포3동,183,5537,3.305
3,1168069000,강남구,개포4동,188,8563,2.195
4,1168052100,강남구,논현1동,236,9871,2.391
...,...,...,...,...,...,...
441,1126059000,중랑구,상봉2동,83,5594,1.484
442,1126068000,중랑구,신내1동,265,12917,2.052
443,1126069000,중랑구,신내2동,138,6899,2.000
444,1126060000,중랑구,중화1동,86,5441,1.581


## 생활인구
- 급속충전이 가장 많은 10시~ 19시 생활인구

In [264]:
population_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/MillionVolt/전처리_생활인구.csv')
population_df = population_data.copy()
population_df = population_df[population_df['시간대구분'].isin([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])]
population_df = population_df[['행정동코드','20세~29세', '30세~39세', '40세~49세', '50세~59세', '60세~69세']]
population_df = population_df.groupby('행정동코드')[['20세~29세', '30세~39세', '40세~49세', '50세~59세', '60세~69세']].sum()
population_df.reset_index()


,행정동코드,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세
0,1111051500,1.333966e+07,1.352725e+07,1.459654e+07,1.243762e+07,8.232479e+06
1,1111053000,2.644847e+07,3.632882e+07,3.493107e+07,2.610349e+07,1.353852e+07
2,1111054000,6.877739e+06,6.778119e+06,6.636008e+06,5.492588e+06,3.355768e+06
3,1111055000,9.410403e+06,7.183339e+06,9.312040e+06,9.298410e+06,6.835781e+06
4,1111056000,7.709439e+06,7.866997e+06,1.080814e+07,1.295943e+07,1.061114e+07
...,...,...,...,...,...,...
419,1174065000,1.349761e+07,1.687752e+07,1.616768e+07,1.568422e+07,1.274314e+07
420,1174066000,1.550924e+07,1.738704e+07,2.052343e+07,1.871396e+07,1.465300e+07
421,1174068500,2.913393e+07,3.681104e+07,3.924275e+07,3.961127e+07,3.306074e+07
422,1174069000,1.655605e+06,2.208710e+06,3.075624e+06,3.173240e+06,2.346966e+06


In [265]:
#전치가수 & 생활인구 병합
merge_df_1 = pd.merge(car_df, population_df, on = '행정동코드', how = 'left')

def fill_na_1(df) : 
    mean_20 = df['20세~29세'].mean()
    mean_30 = df['30세~39세'].mean()
    mean_40 = df['40세~49세'].mean()
    mean_50 = df['50세~59세'].mean()
    mean_60 = df['60세~69세'].mean()
    #결측치 대치 
    df['20세~29세'].fillna(mean_20, inplace=True)
    df['30세~39세'].fillna(mean_30, inplace=True)
    df['40세~49세'].fillna(mean_40, inplace=True)
    df['50세~59세'].fillna(mean_50, inplace=True)
    df['60세~69세'].fillna(mean_60, inplace=True)
    
fill_na_1(merge_df_1)
merge_df_1.isna().sum()

행정동코드      0
자치구        0
행정동        0
전기차수       0
등록차대수      0
전기차비율      0
20세~29세    0
30세~39세    0
40세~49세    0
50세~59세    0
60세~69세    0
dtype: int64

In [266]:
merge_df_1

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,5.395397e+06
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,1.411787e+07
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07
3,1168069000,강남구,개포4동,188,8563,2.195,8.244242e+06,1.134663e+07,1.265612e+07,1.176716e+07,7.811696e+06
4,1168052100,강남구,논현1동,236,9871,2.391,5.444380e+07,6.358521e+07,4.698145e+07,2.732283e+07,1.461065e+07
...,...,...,...,...,...,...,...,...,...,...,...
421,1126059000,중랑구,상봉2동,83,5594,1.484,1.937036e+07,2.117034e+07,1.743877e+07,1.923243e+07,1.449809e+07
422,1126068000,중랑구,신내1동,265,12917,2.052,1.476745e+07,1.900252e+07,2.253541e+07,2.221270e+07,1.700123e+07
423,1126069000,중랑구,신내2동,138,6899,2.000,7.431454e+06,9.585409e+06,1.074399e+07,1.127386e+07,9.310556e+06
424,1126060000,중랑구,중화1동,86,5441,1.581,6.223155e+06,7.732878e+06,7.797224e+06,8.586156e+06,7.335723e+06


## 교통량

In [267]:
traffic_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/MillionVolt/전처리_충전수요데이터.csv')
traffic_df = traffic_data.copy()
traffic_df = traffic_df[['행정동코드', '교통량평균']]
traffic_df

,행정동코드,교통량평균
0,1111051500,507.767059
1,1111053000,909.251765
2,1111056000,818.247059
3,1111058000,1315.035294
4,1111061500,946.500000
...,...,...
421,1130560800,1319.914873
422,1130561500,1319.914873
423,1130562500,1319.914873
424,1130563500,1319.914873


In [268]:
#교통량 데이터 병합
merge_df_2 = pd.merge(merge_df_1, traffic_df, on = '행정동코드', how = 'left')
merge_df_2[merge_df_2['교통량평균'].isna()]

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세,교통량평균
16,1168001100,강남구,압구정동,293,11936,2.455,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
21,1168050200,강남구,청담동,296,11609,2.550,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
86,1138063000,관악구,신사동,51,4907,1.039,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
95,1120081000,광진구,광장동,183,11636,1.573,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
96,1120085000,광진구,구의1동,94,7025,1.338,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
97,1120086000,광진구,구의2동,104,7273,1.430,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
98,1120087000,광진구,구의3동,112,9002,1.244,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
99,1120073000,광진구,군자동,84,4896,1.716,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
100,1120078000,광진구,능동,45,3041,1.480,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN
256,1120050100,성동구,옥수동,233,8775,2.655,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,NaN


In [269]:

# 자치구별 평균 교통량 계산
gn_traffic = traffic_data[traffic_data['자치구'] == '강남구']['교통량평균'].mean()
ga_traffic = traffic_data[traffic_data['자치구'] == '관악구']['교통량평균'].mean()
gj_traffic = traffic_data[traffic_data['자치구'] == '광진구']['교통량평균'].mean()
sd_traffic = traffic_data[traffic_data['자치구'] == '성동구']['교통량평균'].mean()
gd_traffic = traffic_data[traffic_data['자치구'] == '강동구']['교통량평균'].mean()

# 대치 부분 수정
merge_df_2.loc[merge_df_2['자치구'] == '강남구', '교통량평균'] = merge_df_2.loc[merge_df_2['자치구'] == '강남구', '교통량평균'].fillna(gn_traffic)
merge_df_2.loc[merge_df_2['자치구'] == '관악구', '교통량평균'] = merge_df_2.loc[merge_df_2['자치구'] == '관악구', '교통량평균'].fillna(ga_traffic)
merge_df_2.loc[merge_df_2['자치구'] == '광진구', '교통량평균'] = merge_df_2.loc[merge_df_2['자치구'] == '광진구', '교통량평균'].fillna(gj_traffic)
merge_df_2.loc[merge_df_2['자치구'] == '성동구', '교통량평균'] = merge_df_2.loc[merge_df_2['자치구'] == '성동구', '교통량평균'].fillna(sd_traffic)
merge_df_2.loc[merge_df_2['자치구'] == '강동구', '교통량평균'] = merge_df_2.loc[merge_df_2['자치구'] == '강동구', '교통량평균'].fillna(sd_traffic)


print(merge_df_2[merge_df_2['교통량평균'].isna()])
merge_df_2

Empty DataFrame
Columns: [행정동코드, 자치구, 행정동, 전기차수, 등록차대수, 전기차비율, 20세~29세, 30세~39세, 40세~49세, 50세~59세, 60세~69세, 교통량평균]
Index: []


,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세,교통량평균
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,5.395397e+06,1890.734902
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,1.411787e+07,1890.734902
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,2696.682353
3,1168069000,강남구,개포4동,188,8563,2.195,8.244242e+06,1.134663e+07,1.265612e+07,1.176716e+07,7.811696e+06,1890.734902
4,1168052100,강남구,논현1동,236,9871,2.391,5.444380e+07,6.358521e+07,4.698145e+07,2.732283e+07,1.461065e+07,1514.028235
...,...,...,...,...,...,...,...,...,...,...,...,...
421,1126059000,중랑구,상봉2동,83,5594,1.484,1.937036e+07,2.117034e+07,1.743877e+07,1.923243e+07,1.449809e+07,1709.949412
422,1126068000,중랑구,신내1동,265,12917,2.052,1.476745e+07,1.900252e+07,2.253541e+07,2.221270e+07,1.700123e+07,1709.949412
423,1126069000,중랑구,신내2동,138,6899,2.000,7.431454e+06,9.585409e+06,1.074399e+07,1.127386e+07,9.310556e+06,1709.949412
424,1126060000,중랑구,중화1동,86,5441,1.581,6.223155e+06,7.732878e+06,7.797224e+06,8.586156e+06,7.335723e+06,1709.949412


## 충전량

In [270]:
charging_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/서울시 전기차 급속충전기 정보 현황.csv', encoding = 'cp949')
charging_df = charging_data.copy()
charging_df = pd.DataFrame(charging_df.groupby('시군구')['사용량(kWh)'].sum()).reset_index()
charging_df = charging_df.rename(columns = {'시군구' : '자치구'})
charging_df

,자치구,사용량(kWh)
0,강남구,1.378437e+06
1,강동구,4.963556e+05
2,강북구,2.124923e+05
3,강서구,7.444231e+05
4,관악구,5.700365e+05
5,광진구,6.568748e+05
6,구로구,3.997895e+05
7,금천구,3.362122e+05
8,노원구,6.563076e+05
9,도봉구,4.175811e+05


In [271]:
#충전량 데이터 병합
merge_df_3 = pd.merge(merge_df_2,charging_df, on = '자치구', how = 'left')
merge_df_3.head(3)

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세,교통량평균,사용량(kWh)
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,5.395397e+06,1890.734902,1.378437e+06
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,1.411787e+07,1890.734902,1.378437e+06
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,2696.682353,1.378437e+06


## 충전소

In [272]:
charger_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/MillionVolt/전처리_충전소_충전소거리추가.csv')
charger_df = charger_data.copy()
charger_df.head(3)

,설치년도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),충전기ID,충전소ID,위도,경도,법정동코드,법정동명,법정동중심위도,법정동중심경도,충전소중심지거리,인근충전소위도,인근충전소경도,인근충전소거리
0,2017,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174003,37.552909,127.137255,11740107,암사동,37.557805,127.131062,771.170334,37.552708,127.137997,69.126698
1,2017,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174004,37.544812,127.121840,11740109,천호동,37.544376,127.127631,514.128978,37.544789,127.121878,4.227212
2,2017,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174005,37.538795,127.133399,11740109,천호동,37.544376,127.127631,802.154913,37.539128,127.133269,38.707182


In [273]:
import geopandas as gpd

# shapefile을 읽어들임
gdf = gpd.read_file('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/HangJeongDong_ver20230701.geojson')

# 데이터의 구조 확인
gdf.head()

,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
0,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구 사직동,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구 삼청동,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시 종로구 부암동,1101055,1111055000,11110,11,서울특별시,종로구 부암동,종로구,11010550,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시 종로구 평창동,1101056,1111056000,11110,11,서울특별시,종로구 평창동,종로구,11010560,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시 종로구 무악동,1101057,1111057000,11110,11,서울특별시,종로구 무악동,종로구,11010570,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."


In [274]:
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# 행정동 경계 데이터프레임
gdf = gpd.read_file('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/HangJeongDong_ver20230701.geojson')

# tqdm을 사용하여 진행 상황을 표시하면서 charger_df_2의 위도와 경도를 Point 객체로 변환하고 행정동 코드를 가져옴
tqdm.pandas()
charger_df['geometry'] = charger_df.progress_apply(lambda row: Point(row['경도'], row['위도']), axis=1)
charger_df['행정동코드'] = charger_df['geometry'].progress_apply(lambda point: gdf.loc[gdf.contains(point), 'adm_cd2'].iloc[0] if not gdf[gdf.contains(point)].empty else None)


100%|██████████| 45365/45365 [00:20<00:00, 2175.03it/s]


In [275]:
# 결과 확인
charger_df.head(3)

,설치년도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),충전기ID,충전소ID,...,법정동코드,법정동명,법정동중심위도,법정동중심경도,충전소중심지거리,인근충전소위도,인근충전소경도,인근충전소거리,geometry,행정동코드
0,2017,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174003,...,11740107,암사동,37.557805,127.131062,771.170334,37.552708,127.137997,69.126698,POINT (127.1372547 37.5529087),1174057000
1,2017,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174004,...,11740109,천호동,37.544376,127.127631,514.128978,37.544789,127.121878,4.227212,POINT (127.1218396 37.5448122),1174061000
2,2017,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174005,...,11740109,천호동,37.544376,127.127631,802.154913,37.539128,127.133269,38.707182,POINT (127.1333987 37.5387951),1174062000


## 행정동 중심지 거리

In [310]:
hdong_center = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/데이터_서울시_행정동_중심점_2017.csv')
hdong_center = hdong_center.rename(columns = {'코드' : '행정동코드', 'X' : '행정동중심경도', 'Y' : '행정동중심위도'})
hdong_center['행정동코드']=hdong_center['행정동코드'].astype(str)
hdong_center['행정동코드'] = hdong_center['행정동코드'] + '00'
hdong_center

,행정동코드,시도명,시군구명,읍면동명,행정동중심경도,행정동중심위도,Unnamed: 6
0,1111051500,서울특별시,종로구,청운효자동,126.970417,37.584658,NaN
1,1111053000,서울특별시,종로구,사직동,126.970144,37.574108,NaN
2,1111054000,서울특별시,종로구,삼청동,126.981114,37.588013,NaN
3,1111055000,서울특별시,종로구,부암동,126.962557,37.596699,NaN
4,1111056000,서울특별시,종로구,평창동,126.969274,37.613965,NaN
...,...,...,...,...,...,...,...
433,1174052500,서울특별시,강동구,상일제1동,127.164615,37.551513,NaN
434,1153080000,서울특별시,구로구,항동,126.822980,37.484059,NaN
435,1174052600,서울특별시,강동구,상일제2동,127.177116,37.553061,NaN
436,1130563500,서울특별시,강북구,수유3동,127.023089,37.638832,NaN


In [311]:
# 충전소 데이터와 병합
hdong_center['행정동코드'] = hdong_center['행정동코드'].astype(str)
charger_hdong_df = pd.merge(charger_df, hdong_center, on = '행정동코드', how = 'left')
charger_hdong_df.head(3)

,설치년도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),충전기ID,충전소ID,...,인근충전소경도,인근충전소거리,geometry,행정동코드,시도명,시군구명,읍면동명,행정동중심경도,행정동중심위도,Unnamed: 6
0,2017,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174003,...,127.137997,69.126698,POINT (127.1372547 37.5529087),1174057000,서울특별시,강동구,암사제1동,127.134586,37.551855,NaN
1,2017,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174004,...,127.121878,4.227212,POINT (127.1218396 37.5448122),1174061000,서울특별시,강동구,천호제2동,127.122275,37.544691,NaN
2,2017,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174005,...,127.133269,38.707182,POINT (127.1333987 37.5387951),1174062000,서울특별시,강동구,천호제3동,127.133196,37.539654,NaN


In [312]:
#충전소 좌표와 행정동 중심지 거리
import geopy.distance

# 각 충전소와 법정동 중심 간의 최단 거리 계산하여 '충전소중심지거리' 열에 저장
charger_hdong_df['충전소행정동중심거리'] = charger_hdong_df.apply(lambda row: geopy.distance.distance((row['위도'], row['경도']), (row['행정동중심위도'], row['행정동중심경도'])).m, axis=1).round(2)
charger_hdong_df.head(3)


,설치년도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),충전기ID,충전소ID,...,인근충전소거리,geometry,행정동코드,시도명,시군구명,읍면동명,행정동중심경도,행정동중심위도,Unnamed: 6,충전소행정동중심거리
0,2017,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174003,...,69.126698,POINT (127.1372547 37.5529087),1174057000,서울특별시,강동구,암사제1동,127.134586,37.551855,NaN,263.25
1,2017,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174004,...,4.227212,POINT (127.1218396 37.5448122),1174061000,서울특별시,강동구,천호제2동,127.122275,37.544691,NaN,40.77
2,2017,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174005,...,38.707182,POINT (127.1333987 37.5387951),1174062000,서울특별시,강동구,천호제3동,127.133196,37.539654,NaN,96.98


In [313]:
#서울시 충전소만 남기기
charger_hdong_df = charger_hdong_df[charger_hdong_df['시도명']=='서울특별시']

#필요한 컬럼만 남기기
charger_hdong_df = charger_hdong_df[['설치년도', '주소', '충전소명', '시설구분(대)', '시설구분(소)', '기종(대)', '기종(소)',
       '충전기ID', '충전소ID', '위도', '경도', 
       '인근충전소위도', '인근충전소경도', '인근충전소거리', 'geometry', '행정동코드',
       '시군구명', '읍면동명', '행정동중심경도', '행정동중심위도', '충전소행정동중심거리']]
charger_hdong_df

,설치년도,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),충전기ID,충전소ID,위도,...,인근충전소위도,인근충전소경도,인근충전소거리,geometry,행정동코드,시군구명,읍면동명,행정동중심경도,행정동중심위도,충전소행정동중심거리
0,2017,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174003,37.552909,...,37.552708,127.137997,69.126698,POINT (127.1372547 37.5529087),1174057000,강동구,암사제1동,127.134586,37.551855,263.25
1,2017,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174004,37.544812,...,37.544789,127.121878,4.227212,POINT (127.1218396 37.5448122),1174061000,강동구,천호제2동,127.122275,37.544691,40.77
2,2017,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174005,37.538795,...,37.539128,127.133269,38.707182,POINT (127.1333987 37.5387951),1174062000,강동구,천호제3동,127.133196,37.539654,96.98
3,2017,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174006,37.537263,...,37.537235,126.837039,118.174993,POINT (126.8383789 37.5372628),1150054000,강서구,화곡제1동,126.840936,37.533780,447.74
4,2017,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),1,ME174007,37.528956,...,37.529112,126.849078,17.384121,POINT (126.8490887 37.5289561),1150059300,강서구,화곡제8동,126.848633,37.534221,585.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45360,2022,서울특별시 서초구 과천대로 950-18,사당노외공영주차장_서울시,주차시설,공영주차장,완속,AC완속,2,EC001013,37.465563,...,37.462947,126.988464,291.720379,POINT (126.9882261 37.4655634),1165061000,서초구,방배2동,126.988485,37.473015,827.39
45361,2022,서울특별시 서초구 과천대로 950-18,사당노외공영주차장_서울시,주차시설,공영주차장,완속,AC완속,3,EC001013,37.465563,...,37.462947,126.988464,291.720379,POINT (126.9882261 37.4655634),1165061000,서초구,방배2동,126.988485,37.473015,827.39
45362,2022,서울특별시 서초구 헌릉로 170,서초 케이타운오피스텔,공동주택시설,아파트,완속,AC완속,1,NT100137,37.459030,...,37.463000,127.083000,726.920105,POINT (127.0764571 37.4590296),1165066000,서초구,내곡동,127.070548,37.456522,592.28
45363,2022,서울특별시 서초구 헌릉로 170,서초 케이타운오피스텔,공동주택시설,아파트,완속,AC완속,2,NT100137,37.459030,...,37.463000,127.083000,726.920105,POINT (127.0764571 37.4590296),1165066000,서초구,내곡동,127.070548,37.456522,592.28


In [314]:
charger_df_2 = charger_hdong_df.copy()
charger_df_2 = charger_hdong_df[['행정동코드','충전소ID','설치년도','기종(대)', '충전소행정동중심거리']]
charger_df_2

,행정동코드,충전소ID,설치년도,기종(대),충전소행정동중심거리
0,1174057000,ME174003,2017,급속,263.25
1,1174061000,ME174004,2017,급속,40.77
2,1174062000,ME174005,2017,급속,96.98
3,1150054000,ME174006,2017,급속,447.74
4,1150059300,ME174007,2017,급속,585.69
...,...,...,...,...,...
45360,1165061000,EC001013,2022,완속,827.39
45361,1165061000,EC001013,2022,완속,827.39
45362,1165066000,NT100137,2022,완속,592.28
45363,1165066000,NT100137,2022,완속,592.28


In [349]:
charger_count= pd.DataFrame(charger_df_2.groupby('행정동코드')['기종(대)'].count()).reset_index()
fast_charger_count = pd.DataFrame(charger_df_2[charger_df_2['기종(대)']=='급속'].groupby('행정동코드')['기종(대)'].count()).reset_index()
standard_charger_count = pd.DataFrame(charger_df_2[charger_df_2['기종(대)']=='완속'].groupby('행정동코드')['기종(대)'].count()).reset_index()
distance_center_mean = pd.DataFrame(charger_df_2.groupby('행정동코드')['충전소행정동중심거리'].mean().round(2)).reset_index()
distance_center_max = pd.DataFrame(charger_df_2.groupby('행정동코드')['충전소행정동중심거리'].max().round(2)).reset_index()
distance_center_min = pd.DataFrame(charger_df_2.groupby('행정동코드')['충전소행정동중심거리'].min().round(2)).reset_index()

charger_count_df = pd.merge(charger_count, fast_charger_count, on = '행정동코드', how = 'left')
charger_count_df = pd.merge(charger_count_df, standard_charger_count, on = '행정동코드', how = 'left')
charger_count_df = pd.merge(charger_count_df, distance_center_mean, on = '행정동코드', how = 'left')
charger_count_df = pd.merge(charger_count_df, distance_center_max, on = '행정동코드', how = 'left')
charger_count_df = pd.merge(charger_count_df, distance_center_min, on = '행정동코드', how = 'left')


charger_count_df = charger_count_df.rename(columns={'기종(대)_x' : '충전기수','기종(대)_y' :'급속개수','기종(대)' : '완속개수', '충전소행정동중심거리_x' : '행정동중심지평균거리', '충전소행정동중심거리_y' : '행정동중심지최대거리', '충전소행정동중심거리' : '행정동중심지최소거리'})

In [350]:
charger_count_df

,행정동코드,충전기수,급속개수,완속개수,행정동중심지평균거리,행정동중심지최대거리,행정동중심지최소거리
0,1111051500,14,5.0,9,214.04,400.40,80.20
1,1111053000,291,18.0,273,288.13,730.88,43.11
2,1111054000,10,2.0,8,1023.41,1045.61,934.60
3,1111055000,11,1.0,10,515.53,814.67,235.36
4,1111056000,34,2.0,32,975.32,1698.60,174.61
...,...,...,...,...,...,...,...
420,1174065000,36,2.0,34,299.32,526.50,80.11
421,1174066000,26,2.0,24,366.38,653.33,67.67
422,1174068500,77,17.0,60,410.72,893.37,25.93
423,1174069000,5,1.0,4,448.53,448.53,448.53


In [351]:
#병합
merge_df_3['행정동코드'] = merge_df_3['행정동코드'].astype(str)
merge_df_4 = pd.merge(merge_df_3, charger_count_df, on = '행정동코드', how = 'left')
merge_df_4

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세,교통량평균,사용량(kWh),충전기수,급속개수,완속개수,행정동중심지평균거리,행정동중심지최대거리,행정동중심지최소거리
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,5.395397e+06,1890.734902,1.378437e+06,28.0,4.0,24.0,825.80,1024.79,164.50
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,1.411787e+07,1890.734902,1.378437e+06,162.0,3.0,159.0,479.33,776.15,122.09
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,2696.682353,1.378437e+06,85.0,12.0,73.0,407.05,1230.96,25.85
3,1168069000,강남구,개포4동,188,8563,2.195,8.244242e+06,1.134663e+07,1.265612e+07,1.176716e+07,7.811696e+06,1890.734902,1.378437e+06,167.0,12.0,155.0,436.59,841.92,146.07
4,1168052100,강남구,논현1동,236,9871,2.391,5.444380e+07,6.358521e+07,4.698145e+07,2.732283e+07,1.461065e+07,1514.028235,1.378437e+06,72.0,12.0,60.0,489.70,758.76,92.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,1126059000,중랑구,상봉2동,83,5594,1.484,1.937036e+07,2.117034e+07,1.743877e+07,1.923243e+07,1.449809e+07,1709.949412,3.508258e+05,79.0,3.0,76.0,684.97,1096.83,208.21
422,1126068000,중랑구,신내1동,265,12917,2.052,1.476745e+07,1.900252e+07,2.253541e+07,2.221270e+07,1.700123e+07,1709.949412,3.508258e+05,421.0,82.0,339.0,660.26,1546.39,61.50
423,1126069000,중랑구,신내2동,138,6899,2.000,7.431454e+06,9.585409e+06,1.074399e+07,1.127386e+07,9.310556e+06,1709.949412,3.508258e+05,95.0,6.0,89.0,465.95,668.15,177.47
424,1126060000,중랑구,중화1동,86,5441,1.581,6.223155e+06,7.732878e+06,7.797224e+06,8.586156e+06,7.335723e+06,1709.949412,3.508258e+05,16.0,NaN,16.0,547.83,569.20,247.72


In [352]:
merge_df_4 = merge_df_4.fillna(0)
merge_df_4.isna().sum()

행정동코드         0
자치구           0
행정동           0
전기차수          0
등록차대수         0
전기차비율         0
20세~29세       0
30세~39세       0
40세~49세       0
50세~59세       0
60세~69세       0
교통량평균         0
사용량(kWh)      0
충전기수          0
급속개수          0
완속개수          0
행정동중심지평균거리    0
행정동중심지최대거리    0
행정동중심지최소거리    0
dtype: int64

## 면적

In [353]:
area_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/행정구역(동별)_20240424181906.csv', header = 2)
area_df = area_data.copy()
area_df= area_df.rename(columns = {'동별(3)' : '읍면동명'})
area_df['구동'] = area_df['동별(2)'] + area_df['읍면동명']
area_df

,동별(1),동별(2),읍면동명,면적 (k㎡),구성비 (%),행정,법정,소계,소계.1,구동
0,서울시,소계,소계,605.21,100.00,426.0,467,12881.0,96502.0,소계소계
1,서울시,종로구,소계,23.91,3.95,17.0,87,279.0,1475.0,종로구소계
2,서울시,종로구,사직동,1.23,0.20,1.0,12,16.0,68.0,종로구사직동
3,서울시,종로구,삼청동,1.49,0.25,1.0,7,10.0,42.0,종로구삼청동
4,서울시,종로구,부암동,2.27,0.38,1.0,3,18.0,98.0,종로구부암동
...,...,...,...,...,...,...,...,...,...,...
447,서울시,강동구,둔촌1동,0.92,0.15,1.0,1,23.0,229.0,강동구둔촌1동
448,서울시,강동구,둔촌2동,1.56,0.26,1.0,-,29.0,260.0,강동구둔촌2동
449,서울시,강동구,강일동,2.26,0.37,1.0,1,38.0,296.0,강동구강일동
450,서울시,강동구,상일1동,1.82,0.30,1.0,1,46.0,397.0,강동구상일1동


In [354]:
hdong = pdr.code_hdong()
hdong = hdong[hdong['말소일자']==''][hdong['시도명']=='서울특별시']
hdong['구동'] = hdong['시군구명']+hdong['읍면동명']
hdong

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)


/var/folders/tz/h0d7hy211kj12my58s3b3m7m0000gn/T/ipykernel_1484/1827730743.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hdong = hdong[hdong['말소일자']==''][hdong['시도명']=='서울특별시']


,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,생성일자,말소일자,구동
0,11,서울특별시,11000,,1100000000,,19880423,,
1,11,서울특별시,11110,종로구,1111000000,,19880423,,종로구
3,11,서울특별시,11110,종로구,1111051500,청운효자동,20081101,,종로구청운효자동
5,11,서울특별시,11110,종로구,1111053000,사직동,19880423,,종로구사직동
6,11,서울특별시,11110,종로구,1111054000,삼청동,19880423,,종로구삼청동
...,...,...,...,...,...,...,...,...,...
745,11,서울특별시,11740,강동구,1174065000,성내제2동,19880423,,강동구성내제2동
746,11,서울특별시,11740,강동구,1174066000,성내제3동,19880423,,강동구성내제3동
749,11,서울특별시,11740,강동구,1174068500,길동,20080707,,강동구길동
750,11,서울특별시,11740,강동구,1174069000,둔촌제1동,19880423,,강동구둔촌제1동


In [355]:
area_df_2 = pd.merge(area_df, hdong, on = '구동', how = 'left')
area_df_2 = area_df_2[['면적 (k㎡)', '동별(2)','읍면동명_x', '행정동코드']]
area_df_2 = area_df_2[area_df_2['읍면동명_x']!='소계']
area_df_2 = area_df_2.rename(columns = {'면적 (k㎡)' : '면적(km2)', '동별(2)' : '자치구명', '읍면동명_x' : '행정동명'})
area_df_2.to_csv('행정동_면적.csv')

In [356]:
#빈 행정동코드 수기로 채운 후 불러오기
area_df_3 = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/행정동_면적_수정.csv')
area_df_3['행정동코드'] = area_df_3['행정동코드'].astype(str)
area_df_3

,Unnamed: 0,면적(km2),자치구명,행정동명,행정동코드
0,2,1.23,종로구,사직동,1111053000
1,3,1.49,종로구,삼청동,1111054000
2,4,2.27,종로구,부암동,1111055000
3,5,8.87,종로구,평창동,1111056000
4,6,0.36,종로구,무악동,1111057000
...,...,...,...,...,...
423,448,1.56,강동구,둔촌2동,1174070000
424,449,2.26,강동구,강일동,1174051500
425,450,1.82,강동구,상일제1동,1174052500
426,450,1.82,강동구,상일동,1174052000


In [357]:
#면적데이터 병합
merge_df_5 = pd.merge(merge_df_4, area_df_3, on = '행정동코드', how = 'left')
merge_df_5[merge_df_5['행정동명'].isna()]
merge_df_5 = merge_df_5.drop(columns = ['Unnamed: 0', '자치구명', '행정동명'])
merge_df_5.isna().sum()

행정동코드         0
자치구           0
행정동           0
전기차수          0
등록차대수         0
전기차비율         0
20세~29세       0
30세~39세       0
40세~49세       0
50세~59세       0
60세~69세       0
교통량평균         0
사용량(kWh)      0
충전기수          0
급속개수          0
완속개수          0
행정동중심지평균거리    0
행정동중심지최대거리    0
행정동중심지최소거리    0
면적(km2)       0
dtype: int64

In [358]:
merge_df_5

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,60세~69세,교통량평균,사용량(kWh),충전기수,급속개수,완속개수,행정동중심지평균거리,행정동중심지최대거리,행정동중심지최소거리,면적(km2)
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,5.395397e+06,1890.734902,1.378437e+06,28.0,4.0,24.0,825.80,1024.79,164.50,1.27
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,1.411787e+07,1890.734902,1.378437e+06,162.0,3.0,159.0,479.33,776.15,122.09,2.51
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,1.136525e+07,2696.682353,1.378437e+06,85.0,12.0,73.0,407.05,1230.96,25.85,1.24
3,1168069000,강남구,개포4동,188,8563,2.195,8.244242e+06,1.134663e+07,1.265612e+07,1.176716e+07,7.811696e+06,1890.734902,1.378437e+06,167.0,12.0,155.0,436.59,841.92,146.07,1.49
4,1168052100,강남구,논현1동,236,9871,2.391,5.444380e+07,6.358521e+07,4.698145e+07,2.732283e+07,1.461065e+07,1514.028235,1.378437e+06,72.0,12.0,60.0,489.70,758.76,92.25,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,1126059000,중랑구,상봉2동,83,5594,1.484,1.937036e+07,2.117034e+07,1.743877e+07,1.923243e+07,1.449809e+07,1709.949412,3.508258e+05,79.0,3.0,76.0,684.97,1096.83,208.21,0.65
422,1126068000,중랑구,신내1동,265,12917,2.052,1.476745e+07,1.900252e+07,2.253541e+07,2.221270e+07,1.700123e+07,1709.949412,3.508258e+05,421.0,82.0,339.0,660.26,1546.39,61.50,2.56
423,1126069000,중랑구,신내2동,138,6899,2.000,7.431454e+06,9.585409e+06,1.074399e+07,1.127386e+07,9.310556e+06,1709.949412,3.508258e+05,95.0,6.0,89.0,465.95,668.15,177.47,0.93
424,1126060000,중랑구,중화1동,86,5441,1.581,6.223155e+06,7.732878e+06,7.797224e+06,8.586156e+06,7.335723e+06,1709.949412,3.508258e+05,16.0,0.0,16.0,547.83,569.20,247.72,0.64


## 주차장수

In [359]:
parking_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/주차장(동별)(2016년+이후)_20240401141024.csv')
parking_df = parking_data.copy()
parking_df['행정동'] = parking_df['동별(3)'] 
parking_df = parking_df[parking_df['동별(3)']!='소계']
parking_df = parking_df[['공영주차장수', '행정동']]
merge_df_6 = pd.merge(merge_df_5, parking_df, on = '행정동', how = 'left')
merge_df_6

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,...,교통량평균,사용량(kWh),충전기수,급속개수,완속개수,행정동중심지평균거리,행정동중심지최대거리,행정동중심지최소거리,면적(km2),공영주차장수
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,...,1890.734902,1.378437e+06,28.0,4.0,24.0,825.80,1024.79,164.50,1.27,2
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,...,1890.734902,1.378437e+06,162.0,3.0,159.0,479.33,776.15,122.09,2.51,2
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,...,2696.682353,1.378437e+06,85.0,12.0,73.0,407.05,1230.96,25.85,1.24,2
3,1168069000,강남구,개포4동,188,8563,2.195,8.244242e+06,1.134663e+07,1.265612e+07,1.176716e+07,...,1890.734902,1.378437e+06,167.0,12.0,155.0,436.59,841.92,146.07,1.49,4
4,1168052100,강남구,논현1동,236,9871,2.391,5.444380e+07,6.358521e+07,4.698145e+07,2.732283e+07,...,1514.028235,1.378437e+06,72.0,12.0,60.0,489.70,758.76,92.25,1.25,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,1126059000,중랑구,상봉2동,83,5594,1.484,1.937036e+07,2.117034e+07,1.743877e+07,1.923243e+07,...,1709.949412,3.508258e+05,79.0,3.0,76.0,684.97,1096.83,208.21,0.65,34
424,1126068000,중랑구,신내1동,265,12917,2.052,1.476745e+07,1.900252e+07,2.253541e+07,2.221270e+07,...,1709.949412,3.508258e+05,421.0,82.0,339.0,660.26,1546.39,61.50,2.56,20
425,1126069000,중랑구,신내2동,138,6899,2.000,7.431454e+06,9.585409e+06,1.074399e+07,1.127386e+07,...,1709.949412,3.508258e+05,95.0,6.0,89.0,465.95,668.15,177.47,0.93,5
426,1126060000,중랑구,중화1동,86,5441,1.581,6.223155e+06,7.732878e+06,7.797224e+06,8.586156e+06,...,1709.949412,3.508258e+05,16.0,0.0,16.0,547.83,569.20,247.72,0.64,32


## 비거주지 비율

In [360]:
residential_data = pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/용도지역+현황_20240425121959_수정.csv')
residential_df = residential_data.copy()
residential_df['비주거지비율(구))'] = 100 - ((residential_df['주거지역'] / residential_df['소계']) *100).round(2)
residential_df = residential_df.rename(columns = {'자치구별(2)' : '자치구'})
residential_df

,자치구별(1),자치구,소계,주거지역,비주거지비율(구))
0,서울시,종로구,23972507,9830838,58.99
1,서울시,중구,9974292,6013132,39.71
2,서울시,용산구,21898766,12181023,44.38
3,서울시,성동구,16804426,9931728,40.90
4,서울시,광진구,17075115,11593626,32.10
5,서울시,동대문구,14245433,13036995,8.48
6,서울시,중랑구,18531520,10808521,41.67
7,서울시,성북구,24621432,17826125,27.60
8,서울시,강북구,23636019,10890430,53.92
9,서울시,도봉구,20679052,8095692,60.85


In [361]:
#데이터병합
merge_df_7 = pd.merge(merge_df_6, residential_df, on = '자치구', how = 'left')
merge_df_7 = merge_df_7.drop(columns = ['자치구별(1)', '소계', '주거지역'])
merge_df_7.head(3)

,행정동코드,자치구,행정동,전기차수,등록차대수,전기차비율,20세~29세,30세~39세,40세~49세,50세~59세,...,사용량(kWh),충전기수,급속개수,완속개수,행정동중심지평균거리,행정동중심지최대거리,행정동중심지최소거리,면적(km2),공영주차장수,비주거지비율(구))
0,1168066000,강남구,개포1동,234,10434,2.243,3.468640e+06,5.813829e+06,8.719380e+06,7.172275e+06,...,1.378437e+06,28.0,4.0,24.0,825.80,1024.79,164.50,1.27,2,38.66
1,1168067000,강남구,개포2동,443,15656,2.830,1.215198e+07,1.663985e+07,2.362627e+07,1.878887e+07,...,1.378437e+06,162.0,3.0,159.0,479.33,776.15,122.09,2.51,2,38.66
2,1168067500,강남구,개포3동,183,5537,3.305,1.763513e+07,1.913542e+07,1.897484e+07,1.620236e+07,...,1.378437e+06,85.0,12.0,73.0,407.05,1230.96,25.85,1.24,2,38.66


In [435]:
# '-' 값을 0으로 대체
merge_df_7.loc[access_column['공영주차장수'] == '-', '공영주차장수'] = 0
# 모든 값을 정수형으로 변환
merge_df_7['공영주차장수'] = merge_df_7['공영주차장수'].astype(int)

In [441]:
#차충비 계산
merge_df_7['급속차충비'] = merge_df_7['전기차수'] / merge_df_7['급속개수']
merge_df_7.loc[merge_df_7['급속개수'] == 0, '급속차충비'] = merge_df_7['전기차수']

In [442]:
merge_df_7.to_csv('전처리_행정동_최종.csv')

---
# 분석

In [443]:
merge_df_7.columns

Index(['행정동코드', '자치구', '행정동', '전기차수', '등록차대수', '전기차비율', '20세~29세', '30세~39세',
       '40세~49세', '50세~59세', '60세~69세', '교통량평균', '사용량(kWh)', '충전기수', '급속개수',
       '완속개수', '행정동중심지평균거리', '행정동중심지최대거리', '행정동중심지최소거리', '면적(km2)', '공영주차장수',
       '비주거지비율(구))', '급속차충비'],
      dtype='object')

In [445]:
access_column = merge_df_7[['급속차충비', '급속개수', '행정동중심지평균거리', '공영주차장수']]
access_column

,급속차충비,급속개수,행정동중심지평균거리,공영주차장수
0,58.500000,4.0,825.80,2
1,147.666667,3.0,479.33,2
2,15.250000,12.0,407.05,2
3,15.666667,12.0,436.59,4
4,19.666667,12.0,489.70,7
...,...,...,...,...
423,27.666667,3.0,684.97,34
424,3.231707,82.0,660.26,20
425,23.000000,6.0,465.95,5
426,86.000000,0.0,547.83,32


In [453]:
import numpy as np

# 비교 행렬 작성
comparison_matrix = np.array([[1, 1/5, 1/3, 3],  # 급속차충비와 각 요소의 상대적 중요도 비교
                              [5, 1, 1, 3],    # 급속개수와 각 요소의 상대적 중요도 비교
                              [3, 1, 1, 5],# 행정동중심지평균거리와 각 요소의 상대적 중요도 비교
                              [1/3, 1/3, 1/5, 1]])     # 공영주차장수와 각 요소의 상대적 중요도 비교

# 고유벡터 계산
eigenvalues, eigenvectors = np.linalg.eig(comparison_matrix)

# 최대 고유값에 해당하는 고유벡터 선택
max_eigenvalue_index = np.argmax(eigenvalues)
weights = eigenvectors[:, max_eigenvalue_index]

# 상대적 중요도 정규화
weights_normalized = weights / np.sum(weights)

# 결과 출력 (소수점 2자리까지)
for i, factor in enumerate(access_column.columns):
    print(f"{factor}의 상대적 중요도: {weights_normalized[i]:.2f}")


급속차충비의 상대적 중요도: 0.14-0.00j
급속개수의 상대적 중요도: 0.41-0.00j
행정동중심지평균거리의 상대적 중요도: 0.38-0.00j
공영주차장수의 상대적 중요도: 0.08-0.00j


In [447]:
# 일관성 비율 계산 함수 정의
def consistency_ratio(comparison_matrix):
    n = comparison_matrix.shape[0]
    eigenvalues, _ = np.linalg.eig(comparison_matrix)
    lambda_max = np.max(eigenvalues)
    consistency_index = (lambda_max - n) / (n - 1)
    random_index = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.54, 13: 1.56, 14: 1.58, 15: 1.59}
    random_index_max = random_index[n]
    consistency_ratio = consistency_index / random_index_max
    return consistency_ratio

# 비교 행렬의 일관성 비율 계산
cr = consistency_ratio(comparison_matrix)

# 결과 출력
print("Consistency Ratio:", cr)


Consistency Ratio: (0.08343042145742786+0j)
